# Aviation Accidents Analysis

## 1. Overview

A company I am working with is planning to diversify its portfolio by exploring new areas, particularly the aviation industry. The leadership is interested in the risks associated with aircraft operations, mainly safety and reliability. It is on that background that as a senior data scientist in the company that the management tusked me with the responsibility of carrying out data analysis to assist the management in making informed investment decisions to determine which aircraft presents the lowest operational risk.


## 2. Business Understanding
The company's goal is to identify the safest and most reliable aircraft to purchase based on an analysis of accident and incident data collected over the years. Therefore, the company wants to answer the following questions before taking any steps to make a purchase.
The company wants aircraft:

* That is not accident-prone/low accident frequency.
* If ever involved in an accident, then a low number of deaths and injuries.
* For aircraft that are frequently involved in accidents, who built their engines, and what models are they?
* Are the aircraft that are frequently accident-prone professionally or amateur-built?
* What are the leading causes of accidents? Are the causes attributed to the engine or other parameters, such as weather?

## 3. Data Science Understanding
Analyze accident data collected over the years to establish which aircraft has the lowest risk by using risk indicators such as accident frequency, fatality rate, and severity of injuries, etc.
This will involve the following analysis:
* Data exploration
* Data Cleaning
* Grouping aircraft by make and model
* Calculating the safety matrix per aircraft type
* Visualizing patterns of high-risk and low-risk aircraft


In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Loading accident data to the notebook
aviation_data = pd.read_csv("../Data/AviationData.csv", index_col = 0, encoding = "latin1", low_memory = False)

### 3.1 exploring data

In [3]:
aviation_data.head() # checking the first five rows of the dataframe

,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
Event.Id,,,,,,,,,,,,,,,,,,,,,
20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,Fatal(2),...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,Fatal(4),...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,Fatal(3),...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,Fatal(2),...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,Fatal(1),...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [4]:
aviation_data.tail() # chwcking the last five rows of the dataframe

,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
Event.Id,,,,,,,,,,,,,,,,,,,,,
20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,Minor,...,Personal,NaN,0.0,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022
20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,Non-Fatal,...,Personal,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022
20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,NaN,...,Personal,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
20221230106513,Accident,ERA23LA097,2022-12-29,"Athens, GA",United States,NaN,NaN,NaN,NaN,Minor,...,Personal,NaN,0.0,1.0,0.0,1.0,NaN,NaN,NaN,30-12-2022


In [5]:
aviation_data.info() # getting concise information about the dataframe

<class 'pandas.core.frame.DataFrame'>
Index: 88889 entries, 20001218X45444 to 20221230106513
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Investigation.Type      88889 non-null  object 
 1   Accident.Number         88889 non-null  object 
 2   Event.Date              88889 non-null  object 
 3   Location                88837 non-null  object 
 4   Country                 88663 non-null  object 
 5   Latitude                34382 non-null  object 
 6   Longitude               34373 non-null  object 
 7   Airport.Code            50132 non-null  object 
 8   Airport.Name            52704 non-null  object 
 9   Injury.Severity         87889 non-null  object 
 10  Aircraft.damage         85695 non-null  object 
 11  Aircraft.Category       32287 non-null  object 
 12  Registration.Number     87507 non-null  object 
 13  Make                    88826 non-null  object 
 14  Model                

All the columns have missing values except the first three.

In [6]:
aviation_data.shape # checking the dataframe dimensions, i.e number of rows and cloumns in the dataframe.

(88889, 30)

Dataset contains 88,889 rows and 30 columns.

## 4. Data Preparation and cleaning

### 4.1 Checking for missing values

In [7]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88889 entries, 20001218X45444 to 20221230106513
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Investigation.Type      88889 non-null  object 
 1   Accident.Number         88889 non-null  object 
 2   Event.Date              88889 non-null  object 
 3   Location                88837 non-null  object 
 4   Country                 88663 non-null  object 
 5   Latitude                34382 non-null  object 
 6   Longitude               34373 non-null  object 
 7   Airport.Code            50132 non-null  object 
 8   Airport.Name            52704 non-null  object 
 9   Injury.Severity         87889 non-null  object 
 10  Aircraft.damage         85695 non-null  object 
 11  Aircraft.Category       32287 non-null  object 
 12  Registration.Number     87507 non-null  object 
 13  Make                    88826 non-null  object 
 14  Model                

In [ ]:
aviation_data.isnull().sun()

In [8]:
missing_values_percentage= aviation_data.isnull().mean().sort_values(ascending = False) * 100
missing_values_percentage

Schedule                  85.845268
Air.carrier               81.271023
FAR.Description           63.974170
Aircraft.Category         63.677170
Longitude                 61.330423
Latitude                  61.320298
Airport.Code              43.601570
Airport.Name              40.708074
Broad.phase.of.flight     30.560587
Publication.Date          15.492356
Total.Serious.Injuries    14.073732
Total.Minor.Injuries      13.424608
Total.Fatal.Injuries      12.826109
Engine.Type                7.982990
Report.Status              7.181991
Purpose.of.flight          6.965991
Number.of.Engines          6.844491
Total.Uninjured            6.650992
Weather.Condition          5.053494
Aircraft.damage            3.593246
Registration.Number        1.554748
Injury.Severity            1.124999
Country                    0.254250
Amateur.Built              0.114750
Model                      0.103500
Make                       0.070875
Location                   0.058500
Accident.Number            0

In [9]:
# columns with more than 60 percent missing values
columns_to_drop = missing_values_percentage.head(6) 
columns_to_drop

Schedule             85.845268
Air.carrier          81.271023
FAR.Description      63.974170
Aircraft.Category    63.677170
Longitude            61.330423
Latitude             61.320298
dtype: float64

In [10]:
# Dropping columns that have more than 60 percent missing values
aviation_data = aviation_data.drop(columns= columns_to_drop.index)
aviation_data

,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Registration.Number,...,Engine.Type,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
Event.Id,,,,,,,,,,,,,,,,,,,,,
20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,Fatal(2),Destroyed,NC6404,...,Reciprocating,Personal,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,Fatal(4),Destroyed,N5069P,...,Reciprocating,Personal,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,NaN,NaN,Fatal(3),Destroyed,N5142R,...,Reciprocating,Personal,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,Fatal(2),Destroyed,N1168J,...,Reciprocating,Personal,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,Fatal(1),Destroyed,N15NY,...,NaN,Personal,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,Minor,NaN,N1867H,...,NaN,Personal,0.0,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022
20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,N2895Z,...,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,PAN,PAYSON,Non-Fatal,Substantial,N749PJ,...,NaN,Personal,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022


In [11]:
aviation_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88889 entries, 20001218X45444 to 20221230106513
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Investigation.Type      88889 non-null  object 
 1   Accident.Number         88889 non-null  object 
 2   Event.Date              88889 non-null  object 
 3   Location                88837 non-null  object 
 4   Country                 88663 non-null  object 
 5   Airport.Code            50132 non-null  object 
 6   Airport.Name            52704 non-null  object 
 7   Injury.Severity         87889 non-null  object 
 8   Aircraft.damage         85695 non-null  object 
 9   Registration.Number     87507 non-null  object 
 10  Make                    88826 non-null  object 
 11  Model                   88797 non-null  object 
 12  Amateur.Built           88787 non-null  object 
 13  Number.of.Engines       82805 non-null  float64
 14  Engine.Type          

### 4.2 Filling the missing values

In [12]:
categorical_columns = ["Investigation.Type", "Accident.Number", "Event.Date", "Location", "Country", "Airport.Code", "Airport.Name", "Injury.Severity", "Aircraft.damage", "Registration.Number", "Make", "Model", "Amateur.Built", "Purpose.of.flight", "Weather.Condition", "Broad.phase.of.flight", "Report.Status","Publication.Date"]
categorical_columns

['Investigation.Type',
 'Accident.Number',
 'Event.Date',
 'Location',
 'Country',
 'Airport.Code',
 'Airport.Name',
 'Injury.Severity',
 'Aircraft.damage',
 'Registration.Number',
 'Make',
 'Model',
 'Amateur.Built',
 'Purpose.of.flight',
 'Weather.Condition',
 'Broad.phase.of.flight',
 'Report.Status',
 'Publication.Date']

In [13]:
aviation_data[categorical_columns] = aviation_data[categorical_columns].fillna("unknown")
aviation_data

,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Registration.Number,...,Engine.Type,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
Event.Id,,,,,,,,,,,,,,,,,,,,,
20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,unknown,unknown,Fatal(2),Destroyed,NC6404,...,Reciprocating,Personal,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,unknown
20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,unknown,unknown,Fatal(4),Destroyed,N5069P,...,Reciprocating,Personal,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,unknown,unknown,Fatal(3),Destroyed,N5142R,...,Reciprocating,Personal,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,unknown,unknown,Fatal(2),Destroyed,N1168J,...,Reciprocating,Personal,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,unknown,unknown,Fatal(1),Destroyed,N15NY,...,NaN,Personal,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,unknown,unknown,Minor,unknown,N1867H,...,NaN,Personal,0.0,1.0,0.0,0.0,unknown,unknown,unknown,29-12-2022
20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,unknown,unknown,unknown,unknown,N2895Z,...,NaN,unknown,0.0,0.0,0.0,0.0,unknown,unknown,unknown,unknown
20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,PAN,PAYSON,Non-Fatal,Substantial,N749PJ,...,NaN,Personal,0.0,0.0,0.0,1.0,VMC,unknown,unknown,27-12-2022


In [14]:
aviation_data.sample(10)

,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Registration.Number,...,Engine.Type,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
Event.Id,,,,,,,,,,,,,,,,,,,,,
20210816103709,Accident,ANC21LA070,2021-08-05,"Talkeetna, AK",United States,AK44,TALKEETNA VILLAGE STRIP,Non-Fatal,Substantial,N148CM,...,NaN,Personal,0.0,0.0,0.0,2.0,VMC,unknown,unknown,unknown
20001213X31410,Accident,ATL87LA222,1987-07-23,"HUNTSVILLE, AL",United States,unknown,unknown,Non-Fatal,Substantial,N972B,...,Reciprocating,Aerial Observation,0.0,0.0,2.0,0.0,VMC,Maneuvering,Probable Cause,24-04-1989
20001212X21685,Accident,CHI00LA267,2000-08-21,"NORTH PLATTE, NE",United States,LBF,NORTH PLATTE REGIONAL,Non-Fatal,Substantial,N4256R,...,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Taxi,Probable Cause,21-08-2001
20130808X62519,Accident,CEN13LA472,2013-08-04,"Simmesport, LA",United States,unknown,unknown,Non-Fatal,Substantial,N3655G,...,Reciprocating,Aerial Application,0.0,1.0,0.0,0.0,VMC,unknown,The pilot's failure to maintain proper airplan...,25-09-2020
20020816X01414,Accident,NYC02FA152,2002-08-01,"Marietta, PA",United States,N71,Donegal Springs,Fatal(4),Destroyed,N81310,...,Reciprocating,Personal,4.0,NaN,NaN,NaN,VMC,Takeoff,Probable Cause,16-12-2009
20030711X01074,Accident,ANC03LA069,2003-07-04,"SELDOVIA, AK",United States,SOV,Seldovia,Non-Fatal,Substantial,N63541,...,Reciprocating,Personal,NaN,NaN,NaN,3.0,VMC,Landing,Probable Cause,30-12-2003
20021120X05488,Accident,MIA03LA014,2002-11-10,"FOXWORTH, MS",United States,unknown,Circle Bar Ranch,Non-Fatal,Substantial,N990PT,...,Turbo Prop,Personal,NaN,NaN,NaN,6.0,VMC,Landing,Probable Cause,01-04-2003
20090811X25902,Accident,ERA09LA459,2009-08-12,"Sussex, NJ",United States,FWN,Sussex Airport,Non-Fatal,Substantial,N6676S,...,Reciprocating,Personal,0.0,0.0,2.0,0.0,VMC,unknown,The pilots improper pre-flight planning and t...,25-09-2020
20180302X81813,Accident,WPR18WA097,2018-02-28,"Ogies, South Africa",South Africa,unknown,unknown,Fatal,Destroyed,ZS-HKZ,...,NaN,unknown,2.0,0.0,0.0,0.0,unknown,unknown,unknown,25-09-2020


In [15]:
numericals_columns = ["Total.Fatal.Injuries", "Total.Serious.Injuries", "Total.Minor.Injuries", "Total.Uninjured", "Number.of.Engines"]
numericals_columns

['Total.Fatal.Injuries',
 'Total.Serious.Injuries',
 'Total.Minor.Injuries',
 'Total.Uninjured',
 'Number.of.Engines']

In [16]:
aviation_data[numericals_columns] = aviation_data[numericals_columns].fillna(0)
aviation_data

,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Registration.Number,...,Engine.Type,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
Event.Id,,,,,,,,,,,,,,,,,,,,,
20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,unknown,unknown,Fatal(2),Destroyed,NC6404,...,Reciprocating,Personal,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,unknown
20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,unknown,unknown,Fatal(4),Destroyed,N5069P,...,Reciprocating,Personal,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,unknown,unknown,Fatal(3),Destroyed,N5142R,...,Reciprocating,Personal,3.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,26-02-2007
20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,unknown,unknown,Fatal(2),Destroyed,N1168J,...,Reciprocating,Personal,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,unknown,unknown,Fatal(1),Destroyed,N15NY,...,NaN,Personal,1.0,2.0,0.0,0.0,VMC,Approach,Probable Cause,16-04-1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,unknown,unknown,Minor,unknown,N1867H,...,NaN,Personal,0.0,1.0,0.0,0.0,unknown,unknown,unknown,29-12-2022
20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,unknown,unknown,unknown,unknown,N2895Z,...,NaN,unknown,0.0,0.0,0.0,0.0,unknown,unknown,unknown,unknown
20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,PAN,PAYSON,Non-Fatal,Substantial,N749PJ,...,NaN,Personal,0.0,0.0,0.0,1.0,VMC,unknown,unknown,27-12-2022


In [17]:
aviation_data.sample(10)

,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Registration.Number,...,Engine.Type,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
Event.Id,,,,,,,,,,,,,,,,,,,,,
20001212X19658,Accident,ANC99FA130,1999-09-03,"BETTLES, AK",United States,unknown,unknown,Fatal(1),Destroyed,N9166K,...,Reciprocating,Unknown,1.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,09-05-2001
20001212X22150,Accident,FTW01FA013,2000-10-25,"OKLAHOMA CITY, OK",United States,OKC,Will Rogers World,Fatal(1),Destroyed,N9485T,...,Reciprocating,Business,1.0,0.0,0.0,0.0,IMC,Takeoff,Probable Cause,30-05-2003
20010618X01189,Accident,FTW01LA143,2001-06-14,"OKMULGEE, OK",United States,OKM,Okmulgee Municipal Airport,Non-Fatal,Substantial,N114BW,...,Reciprocating,Personal,0.0,0.0,1.0,0.0,VMC,Cruise,Probable Cause,01-07-2002
20001214X39650,Accident,LAX84LA312,1984-05-21,"HAYWARD, CA",United States,HWD,HAYWARD AIR TERMINAL,Non-Fatal,Substantial,N734TH,...,Reciprocating,Personal,0.0,0.0,0.0,2.0,VMC,Landing,Probable Cause,unknown
20130305X21219,Accident,WPR13LA137,2013-03-02,"Latangai Island, PO",Papua New Guinea,unknown,unknown,Fatal,Substantial,N471M,...,Turbo Shaft,Aerial Observation,1.0,0.0,1.0,0.0,VMC,unknown,The failure of both the pilot and the fish mas...,25-09-2020
20001213X34127,Accident,CHI86LA180,1986-07-25,"DULUTH, MN",United States,unknown,unknown,Non-Fatal,Substantial,N5499B,...,Reciprocating,Instructional,0.0,0.0,0.0,1.0,VMC,Cruise,Probable Cause,unknown
20001212X19270,Accident,FTW99LA189,1999-07-14,"SPRINGTOWN, TX",United States,NONE,KEZER AIR RANCH,Non-Fatal,Substantial,N44677,...,Reciprocating,Personal,0.0,0.0,2.0,0.0,VMC,Takeoff,Probable Cause,30-11-2000
20090724X05537,Accident,ERA09FA417,2009-07-24,"Boonsboro, MD",United States,unknown,NONE,Fatal,Substantial,N7189W,...,Reciprocating,Business,4.0,0.0,0.0,0.0,VMC,unknown,The pilots improper decision to depart on a v...,25-09-2020
20001213X35425,Accident,NYC87LA046,1986-12-07,"TRENTON, ME",United States,unknown,BAR HARBOR,Non-Fatal,Substantial,N7651R,...,Reciprocating,Personal,0.0,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,unknown


### 4.3 Checking for Duplicates

In [18]:
duplicates = aviation_data[aviation_data.duplicated()]
print(len(duplicates))
duplicates.head()

0


,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Registration.Number,...,Engine.Type,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
Event.Id,,,,,,,,,,,,,,,,,,,,,


### 4.4 Checking for extraneouse values

In [19]:
for col in aviation_data.columns:
    print(col, '\n', aviation_data[col].value_counts(normalize=True).head(), '\n\n')

Investigation.Type 
 Investigation.Type
Accident    0.956418
Incident    0.043582
Name: proportion, dtype: float64 


Accident.Number 
 Accident.Number
CEN22LA149    0.000022
WPR23LA041    0.000022
WPR23LA045    0.000022
DCA22WA214    0.000022
DCA22WA089    0.000022
Name: proportion, dtype: float64 


Event.Date 
 Event.Date
1984-06-30    0.000281
1982-05-16    0.000281
2000-07-08    0.000281
1983-08-05    0.000270
1984-08-25    0.000270
Name: proportion, dtype: float64 


Location 
 Location
ANCHORAGE, AK      0.004882
MIAMI, FL          0.002250
ALBUQUERQUE, NM    0.002205
HOUSTON, TX        0.002171
CHICAGO, IL        0.002070
Name: proportion, dtype: float64 


Country 
 Country
United States     0.925289
Brazil            0.004207
Canada            0.004039
Mexico            0.004027
United Kingdom    0.003870
Name: proportion, dtype: float64 


Airport.Code 
 Airport.Code
unknown    0.436016
NONE       0.016740
PVT        0.005456
APA        0.001800
ORD        0.001676
Name: pro

In [20]:
# confirming if there are still missing values in the data
aviation_data.isnull().sum().sort_values(ascending=False).head(10)

Engine.Type               7096
Investigation.Type           0
Accident.Number              0
Report.Status                0
Broad.phase.of.flight        0
Weather.Condition            0
Total.Uninjured              0
Total.Minor.Injuries         0
Total.Serious.Injuries       0
Total.Fatal.Injuries         0
dtype: int64

There are 7096 missing values in Engine. Type columns.

In [21]:
# Handling the remaining missing values in the Engine.Type column

aviation_data['Engine.Type']= aviation_data['Engine.Type'].fillna("Unknown")
aviation_data


,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Registration.Number,...,Engine.Type,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
Event.Id,,,,,,,,,,,,,,,,,,,,,
20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,unknown,unknown,Fatal(2),Destroyed,NC6404,...,Reciprocating,Personal,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,unknown
20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,unknown,unknown,Fatal(4),Destroyed,N5069P,...,Reciprocating,Personal,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,unknown,unknown,Fatal(3),Destroyed,N5142R,...,Reciprocating,Personal,3.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,26-02-2007
20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,unknown,unknown,Fatal(2),Destroyed,N1168J,...,Reciprocating,Personal,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,unknown,unknown,Fatal(1),Destroyed,N15NY,...,Unknown,Personal,1.0,2.0,0.0,0.0,VMC,Approach,Probable Cause,16-04-1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,unknown,unknown,Minor,unknown,N1867H,...,Unknown,Personal,0.0,1.0,0.0,0.0,unknown,unknown,unknown,29-12-2022
20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,unknown,unknown,unknown,unknown,N2895Z,...,Unknown,unknown,0.0,0.0,0.0,0.0,unknown,unknown,unknown,unknown
20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,PAN,PAYSON,Non-Fatal,Substantial,N749PJ,...,Unknown,Personal,0.0,0.0,0.0,1.0,VMC,unknown,unknown,27-12-2022


In [22]:
aviation_data.isnull().sum().sort_values(ascending=False)

Investigation.Type        0
Accident.Number           0
Report.Status             0
Broad.phase.of.flight     0
Weather.Condition         0
Total.Uninjured           0
Total.Minor.Injuries      0
Total.Serious.Injuries    0
Total.Fatal.Injuries      0
Purpose.of.flight         0
Engine.Type               0
Number.of.Engines         0
Amateur.Built             0
Model                     0
Make                      0
Registration.Number       0
Aircraft.damage           0
Injury.Severity           0
Airport.Name              0
Airport.Code              0
Country                   0
Location                  0
Event.Date                0
Publication.Date          0
dtype: int64

In [23]:
aviation_data.to_csv("cleaned_data.csv", index=False)